In [1]:
import flash
from Bio import SeqIO
from ortools.linear_solver import pywraplp
from common import Gene, Target

In [2]:
input_file = '../../dashit_data/human/RN45s.fasta'

In [3]:
#class Gene:
#    def __init__(self, seq):
#        self.seq = seq
#        self.crispr_sites = []
#        
#    def add_crispr_site(self, i, direction):
#        self.crispr_sites.append((i, direction))
#
#    def get_crispr_site(self, i, direction):
#        if direction == 'F':
#            return self.seq[i:i+20]
#        elif direction == 'R':
#            return flash.reverse_complement(self.seq[i+3:i+23])
        
    
        
def read_genes_from_file(filename):
    sequences = SeqIO.parse(open(filename), 'fasta')

    genes = []
    
    for sequence in sequences:
        new_gene = Gene('hi')
        new_gene.seq = sequence.seq
        new_gene.targets = []
           
        for i in flash.kmers_range(new_gene.seq, 23):
            if 'G' == new_gene.seq[i+21] == new_gene.seq[i+22]:
                new_gene.targets.append(Target(flash.forward_20mer_at(new_gene.seq, i, 'F'),
                                               flash.cut_location((i, 'F'))))
            if 'C' == sequence.seq[i] == sequence.seq[i+1]:
                new_gene.targets.append(Target(flash.forward_20mer_at(new_gene.seq, i, 'R'),
                                               flash.cut_location((i, 'R'))))

                genes.append(new_gene)
    return genes

In [4]:
genes = read_genes_from_file(input_file)

hi  is missing a fasta file.


In [5]:
gene = genes[0]
gene.display_gene_cuts()

Need to cut gene first.


0

In [6]:
# Create solver & variables
solver = pywraplp.Solver('SolveIntegerProblem',
                            pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

objective = solver.Objective()

site_variables = []

for i, t in enumerate(gene.targets):
    site_variables.append(solver.IntVar(0, 1, 'x_{}'.format(i)))
    objective.SetCoefficient(site_variables[-1], 1)

objective.SetMinimization()

In [7]:
type(solver)

ortools.linear_solver.pywraplp.Solver

In [26]:
def create_minimum_spacing_constraints(gene, min_spacing, coverage_width, solver):
    # The constraint here is x_i + x_j <= 1
    coverage_constraints = set()
    spacing_constraints = set()
    
    for i, (seq, cut_loc) in enumerate(gene.targets):        
        nearby_sites = [ j for j, (s, l) in enumerate(gene.targets) if l != cut_loc and abs(l - cut_loc) < min_spacing ]
        
        if len(nearby_sites) > 0:
            spacing_constraints.add(tuple(solver.LookupVariable('x_{}'.format(j)) for j in nearby_sites + [i]))

        nearby_sites = [ j for j, (s, l) in enumerate(gene.targets) if l != cut_loc and abs(l - cut_loc) < coverage_width ]

        if len(nearby_sites) > 0:
            coverage_constraints.add(tuple(solver.LookupVariable('x_{}'.format(j)) for j in nearby_sites + [i]))
            
    for constraint in spacing_constraints:
        new_constraint = solver.Constraint(0, 1)
        for variable in constraint:
            new_constraint.SetCoefficient(variable, 1)
            
    for constraint in coverage_constraints:
        new_constraint = solver.Constraint(1, solver.Infinity())
        for variable in constraint:
            new_constraint.SetCoefficient(variable, 1)            
            
    return spacing_constraints, coverage_constraints

In [27]:
hi, bye = create_minimum_spacing_constraints(gene, 50, 200, solver)

In [28]:
result_status = solver.Solve()

In [76]:
result_status

0

In [19]:
solver.Objective().Value()

34.0

In [81]:
site_variables[3].solution_value()

0.0

In [33]:
gene.cuts = []
for i, v in enumerate(site_variables):
    if v.solution_value() > 0.0:
        gene.cuts.append(gene.targets[i])
        
gene.generate_fragments_from_cuts()

In [34]:
gene.display_gene_cuts()

Mutation_ranges were never set.

GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCCTCCGGATGCGCGCGGGGCTCTGGCCTACCGGTGACCCGGCTAGCCGGCCGCGCT
CCTGCTTGAGCCGCCTGCCGGGGCCCGCGGGCCTGCTGTTCTCTCGCGCGTCCGAGCGTCCCGACT|CCCGGTGCCGGCCCGGGTCCGGGTCTCTGACCC
ACCCGGGGGCGGCGGGGAAGGCGGCGAGGGCCACCGTGCCCCCGTGCGCTCTCCGCTGCGGGCGCCCGGGGCGGCCGCGACAACCCCACCCCGCTGGCTC
CGTGCCGTGCGTGTCAGGCGTTCTCGTCTCCGCGGGGTTGTCCGCCGCCCCTTCCCCGGAGTGGGGGGTTGGCCGGAGCCGATCGGCTCGCTGGCCGGCC
GGCCGGCCTCCGCTCCCGGGGGGCTCTTCGTGATCGATGTGGTGACGTCGTGCTCTCCCGGGCCGGGTCCGAGCCGCGACGGGCGAGGGGCGGACGTTCG
TGGCGAACGGGACCGTCCTTCTCGCTCCGCCCCGC|GGGGGTCCCCTCGTCTCTCCTCTCCCCGCCCGCCGGCGGTGCGTGTGGGAAGGCGTGGGGTGCG
GACCCCGGCCCGACCTCGCCGTCCCGCCCGCCGCCTTCTGCGTCGCGGGGCGGGCCGGCGGGGTCCTCTGACGCGGCAGACAGCCCTCGCTGTCGCCTCC
AGTGGTTGTCGACTTGCGGGCGGCCCCCCTCCGCGGCGGTGGGGGTGCCGTCCCGCCGGCCCGTCGTGCTGCCCTCTCGGGGGGTTTGCGCGAGCGTCGG
CTCCGCCTGGGCCCTTGCGGTGCTCCTGGAGCGCTCCGGGTTGTCCCTCAGGTGCCCGAGGCCGAACGGTGGTGTGTCGTTCCCGCCCCCGGCGCCCCCT
CCTCCGGTCGCCGCCGCGGTGTCCGCGCGTGGGTC|CTGAGGGAGCTCGTCGGTGTGG

CGCCCCTTGCCTCTCGGCGCCCCCTCGATGCTCTTAGCTGAGTGTCCCGCGGGGCCCGAAGCGTTTACTTTGAAAAAATTAGAGTGTTCAA|AGCAGGCC
CGAGCCGCCTGGATACCGCAGCTAGGAATAATGGAATAGGACCGCGGTTCTATTTTGTTGGTTTTCGGAACTGAGGCCATGATTAAGAGGGACGGCCGGG
GGCATTCGTATTGCGCCGCTAGAGGTGAAATTCTTGGACCGGCGCAAGACGGACCAGAGCGAAAGCATTTGCCAAGAATGTTTTCATTAATCAAGAACGA
AAGTCGGAGGTTCGAAGACGATCAGATACCGTCGTAGTTCCGACCATAAACGATGCCGACCGGCGATGCGGCGGCGTTATTCCCATGACCCGCCGGGCAG
CTTCCGGGAAACCAAAGTCTTTGGGTTCCGGGGGGAGTATGGTTGCAAAGCTGAAACTTAAAGGAATTGACGGAAGGGCACCACCAGGA|GTGGAGCCTG
CGGCTTAATTTGACTCAACACGGGAAACCTCACCCGGCCCGGACACGGACAGGATTGACAGATTGATAGCTCTTTCTCGATTCCGTGGGTGGTGGTGCAT
GGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAATTCCGATAACGAACGAGACTCTGGCATGCTAACTAGTTACGCGACCCCCGAGCGGTCGGCGTC
CCCCAACTTCTTAGAGGGACAAGTGGCGTTCAGCCACCCGAGATTGAGCAATAACAGGTCTGTGATGCCCTTAGATGTCCGGGGCTGCACGCGCGCTACA
CTGACTGGCTCAGCGTGTGCCTACCCTACGCCGGCAGGCGCGGGTAACCCGTT|GAACCCCATTCGTGATGGGGATCGGGGATTGCAATTATTCCCCATG
AACGAGGAATTCCCAGTAAGTGCGGGTCATAAGCTTGCGTTGATTAAGTCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGATTGG

CGCGAGCCGGGCCCTTCCCGTGGATCGCCCCAGCTGCGGCGGGCGTCGCGGCCGCCCCCGGGGAGCCCGGCGGGCGCCGGCGCGCCCCCCCCCCCACCCC
ACGTCTCGTCGCGCGCGCGTCCGCTGGGGGCGGGGAGCGGTCGGGCGGCGGCGGTCGGCGGGCGGCGGGGCGGGGCGGTTCGTCCCCCCGCCCTACCCCC
CCGGCCCCGTCCGCCCCCCGTTCCCCCCTCCTCCTCGGCGCGCGGCGGCGGCGGCGGCAGGCGGCGGAGGGGCCGCGGGCCGGTCCCCCCCGCCGGGTCC
GCCCCCGGGGCCGCGGTTCCGCGCGGCGCCTCGCCTCGGCCGGCGCCTAGCAGCCGACTTAGAACTGGTGCGG|ACCAGGGGAATCCGACTGTTTAATTA
AAACAAAGCATCGCGAAGGCCCGCGGCGGGTGTTGACGCGATGTGATTTCTGCCCAGTGCTCTGAATGTCAAAGTGAAGAAATTCAATGAAGCGCGGGTA
AACGGCGGGAGTAACTATGACTCTCTTAAGGTAGCCAAATGCCTCGTCATCTAATTAGTGACGCGCATGAATGGATGAACGAGATTCCCACTGTCCCTAC
CTACTATCCAGCGAAACCACAGCCAAGGGAACGGGCTTGGCGGAATCAGCGGGGAAAGAAGACCCTGTTGAGCTTGACTCTAGTCTGGCACGGTGAAGAG
ACATGAGAGGTGTAGAATAAGTGGGAGGCCCCCGGCGCCCCCCCGGTGTCCCCGCGAGGGGCCCGG|GGCGGGGTCCGCCGGCCCTGCGGGCCGCCGGTG
AAATACCACTACTCTGATCGTTTTTTCACTGACCCGGTGAGGCGGGGGGGCGAGCCCCGAGGGGCTCTCGCTTCTGGCGCCAAGCGCCCGGCCGCGCGCC
GGCCGGGCGCGACCCGCTCCGGGGACAGTGCCAGGTGGGGAGTTTGACTGGGGCGGTACACCTGTCAAACGGTAACGCAGGTGTCCTAAGG

CCGAGCCGGGCCCGTGGCCCGCCGGTCCCCGTCCCGGGGGTTGGCCGCGCGGGCCCCGGTGGGGCGGCCACCCGGGGTCCCGGCCCTCGCG


1

In [35]:
gene.display_gene_targets()

Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation

Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation_ranges were never set.
Mutation

GGG|T|CGGTGGCGCCCC|G|C||G|T||GGG|G|CCCGGT|G|GG|CTT|C|CCGGA|G||G|G|T|TCCG|GGG|GTCGG|CCTGCG|GCG|C|G|T|
GCG|GGGGAGG|AGACGG|T|T|C|CGGGG||GACCGGCC|GCG|A|C|TGC|GGCG|GCGG|T|G|G|TGGGGGCAG|C|C|GCGG|GGATC|G|CCGA
|GG|GC|CGGTC|GGCC|G|CCC|CGG|G|T||GCCGCGC|GGTG|CCG|CCG||GCG|G|CGGTGAGGCCCCGC|G|C|GT|GTGTCCCG|GC|C||GC
GG|TCGGCCGCG|C||T|CGAGGGGTC|CCCGT|G|G|CGTCCCCTT|C||C|CCGC|C|G|GCC|GCCT|TTC|TCGCGCCTTCC|CCGT||C|G|CCC
CGG|C|C|T|CGC|CCGTG|G|TCTCTCGTCTTC|TCCCGGC|C|CGCT|C|T|TC|CGAAC||CGGGT|CGGCGCGT|C|CCCCG|G|G|T|GCGCCTC
G|C||TTCCCGGG|C|C|TGCC|GCGG|CCCTT|C|C|CCGA|G|G|CGT|C|CGTCC|CGG|G|C|GT|C|GG|CGTCGGGGAGAGCCCGTC|C|TCC|
C|CGC|G|T|GGCGTCGCCC|CGT|T|C|GGCGCGCGCGTGCGCCCGA|G|C|G|CGG|CCCGGT|G|GTC|CCTGC||C|GGAC|AGGCGTTCGTGCGA
C|GTGTG|G|CGTGGGTCGACCTCCG|CC|T|TGC|CGGTC|GCTCGCCCTT|T||C|CC|C|G|G|||G|T||CG|G|GGGGTGG|GG|CCCG|G|G||
C|CGG|GG|CCTC|GG|CCCCG|G|T|C|GCGGTCCCCC|G||T|C|||C|||CGG|G||C|GGGGGCGGGC|GCGCC|GGCC|G|GCC||TCGG|TCGG
CCCT|CC|C|TTG|G|C|CGTCG|TGTGGCGTGTGCCACCC|CTG|C|G|CCCGC|G|C|C|CG|C|C|G|GC|G|GGGCTC|GG|AGCCG

G|A|GGGGGGCGC|GCCCGGC|T|GAG|A|G|AGA|CG|GGGA|GGGCGGCGCCGCCG|CCG|CCC|G|CG|A|AG|A|CGGAGAGGGAAAGAGAG|AGC
CG|G|CT|CGGGCCGAGT|T|CCCGTG|G|CCGCC|G|CCT|G|C|G|GTCCGGGT|TCCT|C|C|C|||TCG|G|GGGGCTCCCTCG|C|GCC|GCGC|
|GC|GG|CTCGG|G|GT|TCGGGGTT|CGTCGG|CCCC|G|G|C||C|G|GGT|GGAAGGTCCCGTG|C|CCGTC|G|TCGTCGTCGTCGTCGCGCGT|C
GT|CGG|C|G|G|TGGGGGCGTGTTGC|GTGCG|GTG|TGG|T|G|G|TGG|GGG|AGGA|GGA|A|GGCGG|GTCC|G|G|AA|G|G|GGAAGG|GTGC
|C|G|GCG|GGG|A|GAGA|G|G|GT|CGGGGGAGCGCG|TCCCGGT|C||GCCGCGG|TTCGCCGCCC|G|CC||C|C|CG|G|T||G|GCGGCCCG|G
C|G|TCCGGCC|GACC|GCCG|CTC|CCGCG|C|CCCTC|C|T|CCT|CCC|CGC|C|G|CCC|C|TC|C|T|CCG|AGG|CCCCGC|C|C|GTC|C|TC
CT|CGC|CCTCC|C|CGC|G|C|GTACGCGCGCCCGCC||C|GCC|C|GGC|T||CGCCTCGC|GGC|GCGT|C|G|GCCGG|G|G|CCGGGA|GCCCGC
|C|C|CGC|G||G|CCCGCC|C|GGC|C||GCGC|CCG|TG|G|CCGC|G|G||CGCCGGGG|TTCGCGTG|TCCC|CGG|C|G|GC|G|ACCCGCG|G|
GA|CGCCGCGG|TGTCGTCCGCCG|TCG|CGCGCCCG|CC|T|CCG||GCT|CGCGGCCGCGC|CGCGC||C|G|CGC|CGGGG|CCCCGT|C|C|CGAG
|C|T|TC|CG|CGT||C|GG|GGC|GGGGCGGCTCCGCCG|CCG|C|GT|CCTCGG|ACCCGTC|C|CCCC|G||A||C||C||TCCG|CG

TCTAATTAGTGACGCGCAT|GAATGGATGAACGAGATTCCCACT|G|TCCCTAC|C|TACT|ATCCAGCG|AAACCACA|G||CCAAG|G||GAACG|GG
C|TTGGCGGAAT|C|A|GCGGGGAAAGAAGACCCTGT|T|GAGCTTGACTCTA|GTCTG|GCACGGTGAAGAGACAT|GAGAGGTGTAGAAT|A|AGT|G
GGAGGC|CCCCG|G|C|G||CCCCCC|C|G|G|T|G|||TC|CCCGC|G|A||G|G|GGC|C|CG|GG|G|CGGGG|TCCGCCG|GC|C||CTGC|G||G
GCCGCCG|GTG|AAATACCACTA|CTCTGATCGTTTTTTCACTG|ACCCG|GT||G|||A|G|G|CGGGGGGGCGAGCC|C|C|GA|G|G|GGCTCT|CG
CTTCTGGCGCCAAGC||GCCCGGC|C||GCGC||G|CCGGCC|GGGC|GCGACCC|G|C|T|C|CGGGG|AC|AGTGC|C|A|GGT|GGGGAGTT|T|G|
ACTG|GGGCGGTACACCTGTC||AAACGGTA|ACGCAGGTGTC|CTAAG|GCGA|G|CTC|AGGGAGGACAGAAACCTC|CC|GTG|G||A|GCAGAAGG
GCAAAAGCTCGCTTGATCTTGATTTTCAGTACGAATACAGACCGTGA|||A|AGCGGGGCCTCAC|GATCCTTCT|GAC|C|TTTT|GGGTTTT|AAG|C
AGGAGGTGTCAGAAAAGTTAC|C|ACAG|GGA|TAACTGG|CTTG|TGGCGGCCAAGC|GTTCATAGCGACGTCGCTTTTTGATCCTTCG||ATGTCGGC
TCTTCCTATC|ATTGTGAAGCAGAATTCACCAAG|C|GTTGGATTGTTCACCCACT|||A|ATAGGGAACGTG|A|GCTGGGTTTAGACCGTCG|TG|AG
ACAGGTTAGTTTTACCCTAC|T|GATGATGTGTTGTTG|CCATGG|TAATCCTGCT|CAGTA|CGAGAGGAAC|CGCAG|GTTCAGA|CAT

G|G|CC|ACCC|GGG|GTCCCGGCCCTCGCG


1

In [8]:
dir(pywraplp.Solver)

['ABNORMAL',
 'AT_LOWER_BOUND',
 'AT_UPPER_BOUND',
 'Add',
 'BASIC',
 'BOP_INTEGER_PROGRAMMING',
 'BoolVar',
 'CBC_MIXED_INTEGER_PROGRAMMING',
 'CLP_LINEAR_PROGRAMMING',
 'Clear',
 'ComputeConstraintActivities',
 'ComputeExactConditionNumber',
 'Constraint',
 'EnableOutput',
 'ExportModelAsLpFormat',
 'ExportModelAsMpsFormat',
 'FEASIBLE',
 'FIXED_VALUE',
 'FREE',
 'GLOP_LINEAR_PROGRAMMING',
 'INFEASIBLE',
 'Infinity',
 'IntVar',
 'InterruptSolve',
 'Iterations',
 'LoadModelFromProto',
 'LookupConstraint',
 'LookupVariable',
 'Maximize',
 'Minimize',
 'NOT_SOLVED',
 'NumConstraints',
 'NumVar',
 'NumVariables',
 'OPTIMAL',
 'Objective',
 'RowConstraint',
 'SetSolverSpecificParametersAsString',
 'SetTimeLimit',
 'Solve',
 'Sum',
 'SupportsProblemType',
 'SuppressOutput',
 'UNBOUNDED',
 'VerifySolution',
 'WallTime',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash_